# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [19]:
# import libraries
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
import re
import numpy as np
import pandas as pd
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])
import pickle

[nltk_data] Downloading package punkt to /home/ivan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ivan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ivan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/ivan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('messages', 'sqlite:///DisasterResponse.db')  
X = df.message
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    

    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        if tok not in stopwords.words("english"):
            # lemmatize, normalize case, and remove leading/trailing white space
            clean_tok = lemmatizer.lemmatize(tok).strip()
            clean_tokens.append(tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# Spiliting data
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fcebc1bbdc0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def plot_scores(y_test, y_pred):
    #Testing the model
    # Printing the classification report for each label
    i = 0
    for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1
    accuracy = (y_pred == y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))

In [7]:
y_pred = pipeline.predict(X_test)
plot_scores(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.72      0.41      0.53      1525
           1       0.84      0.94      0.89      4982
           2       0.25      0.34      0.29        47

    accuracy                           0.82      6554
   macro avg       0.60      0.57      0.57      6554
weighted avg       0.81      0.82      0.80      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5433
           1       0.83      0.50      0.62      1121

    accuracy                           0.90      6554
   macro avg       0.87      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6531
           1       0.00      0.00      0.00        23

    accuracy                           1.00      6554
   macro avg       

/home/ivan/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ivan/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ivan/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ivan/anaconda3/lib/python3.9/si

              precision    recall  f1-score   support

           0       0.89      0.95      0.92      4682
           1       0.85      0.72      0.78      1872

    accuracy                           0.88      6554
   macro avg       0.87      0.83      0.85      6554
weighted avg       0.88      0.88      0.88      6554

Feature 30: floods
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      5988
           1       0.92      0.48      0.63       566

    accuracy                           0.95      6554
   macro avg       0.94      0.74      0.80      6554
weighted avg       0.95      0.95      0.94      6554

Feature 31: storm
              precision    recall  f1-score   support

           0       0.95      0.98      0.97      5925
           1       0.76      0.54      0.63       629

    accuracy                           0.94      6554
   macro avg       0.86      0.76      0.80      6554
weighted avg       0.93      0.94    

In [8]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7fcebc1bbdc0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7fcebc1bbdc0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
parameters = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 20, 30, 40]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
plot_scores(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.71      0.40      0.51      1525
           1       0.83      0.95      0.89      4982
           2       0.25      0.30      0.27        47

    accuracy                           0.81      6554
   macro avg       0.60      0.55      0.56      6554
weighted avg       0.80      0.81      0.79      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5433
           1       0.84      0.48      0.61      1121

    accuracy                           0.90      6554
   macro avg       0.87      0.73      0.78      6554
weighted avg       0.89      0.90      0.88      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6531
           1       0.00      0.00      0.00        23

    accuracy                           1.00      6554
   macro avg       

/home/ivan/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ivan/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ivan/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ivan/anaconda3/lib/python3.9/si

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      5988
           1       0.92      0.42      0.58       566

    accuracy                           0.95      6554
   macro avg       0.93      0.71      0.78      6554
weighted avg       0.95      0.95      0.94      6554

Feature 31: storm
              precision    recall  f1-score   support

           0       0.95      0.98      0.97      5925
           1       0.74      0.54      0.62       629

    accuracy                           0.94      6554
   macro avg       0.85      0.76      0.79      6554
weighted avg       0.93      0.94      0.93      6554

Feature 32: fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6487
           1       1.00      0.01      0.03        67

    accuracy                           0.99      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.99      0.99      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
pipeline_ada = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [12]:
# Fit the Adaboost Classifier 
pipeline_ada.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fcebc1bbdc0>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [13]:
# Prediction: the Adaboost Classifier 
y_pred = pipeline_ada.predict(X_test)
plot_scores(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.63      0.13      0.22      1525
           1       0.78      0.98      0.87      4982
           2       0.50      0.15      0.23        47

    accuracy                           0.77      6554
   macro avg       0.64      0.42      0.44      6554
weighted avg       0.74      0.77      0.71      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5433
           1       0.78      0.51      0.62      1121

    accuracy                           0.89      6554
   macro avg       0.84      0.74      0.78      6554
weighted avg       0.88      0.89      0.88      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6531
           1       0.11      0.04      0.06        23

    accuracy                           1.00      6554
   macro avg       

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      5988
           1       0.84      0.56      0.67       566

    accuracy                           0.95      6554
   macro avg       0.90      0.78      0.82      6554
weighted avg       0.95      0.95      0.95      6554

Feature 31: storm
              precision    recall  f1-score   support

           0       0.95      0.98      0.96      5925
           1       0.73      0.47      0.57       629

    accuracy                           0.93      6554
   macro avg       0.84      0.73      0.77      6554
weighted avg       0.93      0.93      0.93      6554

Feature 32: fire
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6487
           1       0.61      0.16      0.26        67

    accuracy                           0.99      6554
   macro avg       0.80      0.58      0.63      6554
weighted avg       0.99      0.99      

In [14]:
parameters_ada = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 60, 70]
}

In [15]:
cv_ada = GridSearchCV(pipeline_ada, param_grid = parameters_ada)

In [16]:
cv_ada.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fcebc1bbdc0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 60, 70],
                         'tfidf__use_idf': (True, False)})

In [17]:
y_pred = cv.predict(X_test)
plot_scores(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.71      0.40      0.51      1525
           1       0.83      0.95      0.89      4982
           2       0.25      0.30      0.27        47

    accuracy                           0.81      6554
   macro avg       0.60      0.55      0.56      6554
weighted avg       0.80      0.81      0.79      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5433
           1       0.84      0.48      0.61      1121

    accuracy                           0.90      6554
   macro avg       0.87      0.73      0.78      6554
weighted avg       0.89      0.90      0.88      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6531
           1       0.00      0.00      0.00        23

    accuracy                           1.00      6554
   macro avg       

/home/ivan/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ivan/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ivan/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ivan/anaconda3/lib/python3.9/si

              precision    recall  f1-score   support

           0       0.95      0.98      0.97      5925
           1       0.74      0.54      0.62       629

    accuracy                           0.94      6554
   macro avg       0.85      0.76      0.79      6554
weighted avg       0.93      0.94      0.93      6554

Feature 32: fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6487
           1       1.00      0.01      0.03        67

    accuracy                           0.99      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.99      0.99      0.99      6554

Feature 33: earthquake
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      5912
           1       0.92      0.74      0.82       642

    accuracy                           0.97      6554
   macro avg       0.95      0.87      0.90      6554
weighted avg       0.97      0.97 

### 9. Export your model as a pickle file

In [20]:
file_name = 'classifier.pkl'
with open (file_name, 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.